In [90]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists
from libreco.data import split_by_ratio_chrono, DatasetPure
from libreco.algorithms import ALS
from sklearn.metrics import mean_squared_error
from scipy.sparse import coo_matrix

import importlib
import dataset
importlib.reload(dataset)
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists

In [125]:
data_pd = pd.read_csv('../data/data_train.csv')
data_matrix, _ = extract_matrix_users_movies_ratings(data_pd)
# axis 0 means: we subtract the means for each movie

data_matrix[data_matrix == 0] = 'nan'
MEAN_MATRIX = np.nanmean(data_matrix, axis=0, keepdims=True)

In [126]:
def subtract_mean_extract_lists(pd, users, movies):  
    data_matrix, _ = extract_matrix_users_movies_ratings(pd)
    # axis 0 means: we subtract the means for each movie
    a = np.zeros(len(users))
    data_matrix_normalized = np.subtract(data_matrix, MEAN_MATRIX, where=data_matrix != 0)
    for i, (user, movie) in enumerate(zip(users, movies)):
        a[i] = data_matrix_normalized[user][movie]
    return a

In [127]:
RANDOM_STATE = 58
train_pd, val_pd = train_test_split(data_pd, train_size=0.9, random_state=RANDOM_STATE)

users_train, movies_train, _ = extract_users_movies_ratings_lists(train_pd)
ratings_train = subtract_mean_extract_lists(train_pd, users_train, movies_train)

users_val, movies_val, _ = extract_users_movies_ratings_lists(val_pd)
ratings_val = subtract_mean_extract_lists(val_pd, users_val, movies_val)

test_pd = pd.read_csv('../data/sampleSubmission.csv')
users_test, movies_test, _ = extract_users_movies_ratings_lists(test_pd)

train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
val = pd.DataFrame({'user': users_val, 'item': movies_val, 'label': ratings_val})

# %%
train_data, data_info = DatasetPure.build_trainset(train)
eval_data = DatasetPure.build_evalset(val)

In [115]:
data_pd = pd.read_csv('../data/data_train.csv')
train_pd, val_pd = train_test_split(data_pd, train_size=0.9, random_state=RANDOM_STATE)

users_train, movies_train, ratings_train = subtract_mean_extract_lists(train_pd)
users_train2, movies_train2, ratings_train2 = extract_users_movies_ratings_lists(train_pd)

assert users_train == users_train2
assert movies_train == movies_train2

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [123]:
users_train.shape

(1059256,)

In [122]:
(users_train == np.sort(users_train2)).sum()

1059256

In [128]:
def get_mean(i):
    mean_vector = np.squeeze(MEAN_MATRIX)
    return mean_vector[i]

def get_offset(movies):
    return np.array(list(map(get_mean, movies)))

In [129]:
emb_sizes = [2,4,8,16,32]
regs = [0.1, 0.5, 1, 5, 10]
alphas = [0.01, 0.1, 1, 10, 100]
david = np.zeros((125, len(users_val)))

for i, emb_size in enumerate(emb_sizes):
    for j, reg in enumerate(regs):
        for k, alpha in enumerate(alphas):
            giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                        reg=reg, alpha=alpha, seed=42)
            giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8,
                    eval_data=eval_data, metrics=["rmse", "mae", "r2"])
            david[len(regs)*len(alphas)*i+len(alphas)*j+k] = np.array(giorgio.predict(user=users_val, item=movies_val)) + get_offset(movies_val)
            
mean_squared_error(david.mean(axis=0), ratings_val, squared=False)

Training start time: 2022-07-19 18:17:22
Epoch 1 elapsed: 0.047s


eval_pred: 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 376.00it/s]

	 eval rmse: 1.0376
	 eval mae: 0.8419


	 eval r2: -0.0178
Epoch 2 elapsed: 0.041s


eval_pred: 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 278.51it/s]

...

	 eval rmse: 1.0694
	 eval mae: 0.8398
	 eval r2: -0.0811


Epoch 5 elapsed: 0.487s


eval_pred: 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 190.38it/s]

	 eval rmse: 1.0699
	 eval mae: 0.8391
	 eval r2: -0.0820


4.0079949094124485

In [130]:
mean_squared_error(david.mean(axis=0), ratings_val + get_offset(movies_val), squared=False)

0.9877900619671548

In [87]:
giorgio = ALS(task="rating", data_info=data_info, embed_size=4, n_epochs=5,
                        reg=5, alpha=1, seed=42)
giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8,
                    eval_data=eval_data, metrics=["rmse", "mae", "r2"])
ypred = np.array(giorgio.predict(user=users_val, item=movies_val)) + get_offset(movies_val)
print(mean_squared_error(ypred, ratings_val, squared=False))

Training start time: 2022-07-19 17:31:55
Epoch 1 elapsed: 0.058s


eval_pred: 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 347.66it/s]

	 eval rmse: 4.0174
	 eval mae: 3.8581
	 eval r2: -11.9237


Epoch 2 elapsed: 0.056s


eval_pred: 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 409.79it/s]


...

	 eval rmse: 4.0054
	 eval mae: 3.8517
	 eval r2: -11.8466
Epoch 4 elapsed: 0.054s


eval_pred: 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 395.80it/s]


	 eval rmse: 4.0040
	 eval mae: 3.8521
	 eval r2: -11.8376
Epoch 5 elapsed: 0.073s


eval_pred: 100%|██████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 373.24it/s]


	 eval rmse: 4.0037
	 eval mae: 3.8532
	 eval r2: -11.8357
0.9933771040483091
